# 2. Experiment tracking and model management

## Q1. Install MLflow

Once you installed the package, run the command mlflow --version and check the output.

What's the version that you have?

In [1]:
!mlflow --version

mlflow, version 1.26.1


## Q2. Download and preprocess the data

Your task is to download the datasets and then execute this command:
```
python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output
```
How many files were saved to OUTPUT_FOLDER?

In [7]:
!ls ../processed-data

dv.pkl    test.pkl  train.pkl valid.pkl


There are 4 files.

## Q3. Train a model with autolog

We will train a random forest regressor (from Scikit-Learn) on the taxi dataset.

We have prepared the training script train.py for this exercise, which can be also found in the folder homework. The script will load the datasets produced by the previous step, train the model on the training set and finally calculate the RMSE on the validation set.

Your task is to modify the script to enable autologging with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked.

- Tip 1: don't forget to wrap the training code with a with mlflow.start_run(): statement as we showed in the videos.

- Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

How many parameters are automatically logged by MLflow?

In [17]:
!python train.py --data_path ../processed-data

In [16]:
import mlflow
client = mlflow.tracking.MlflowClient(tracking_uri="sqlite:///mlflow.db")
experiment = client.get_experiment_by_name("homework-autolog")
runs = client.search_runs(experiment.experiment_id)
print(f"There are {len(runs[0].data.params)} parameters logged when using autolog.")

There are 17 parameters logged when using autolog.


## Q4. Launch the tracking server locally

We want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry.

Your task is to

launch the tracking server on your local machine
select a SQLite db for the backend store and a folder called artifacts for the artifacts store
You should keep the tracking server running to work on the next two exercises that use the server.

In addition to backend-store-uri, what else do you need to pass to properly configure the server?

- default-artifact-root
- serve-artifacts
- artifacts-only
- artifacts-destination

In [19]:
!mlflow server --help

Usage: mlflow server [OPTIONS]

  Run the MLflow tracking server.

  The server listens on http://localhost:5000 by default and only accepts
  connections from the local machine. To let the server accept connections
  from other machines, you will need to pass ``--host 0.0.0.0`` to listen on
  all network interfaces (or a specific interface address).

Options:
  --backend-store-uri PATH     URI to which to persist experiment and run
                               data. Acceptable URIs are SQLAlchemy-compatible
                               database connection strings (e.g.
                               'sqlite:///path/to/file.db') or local
                               filesystem URIs (e.g.
                               'file:///absolute/path/to/directory'). By
                               default, data will be logged to the ./mlruns
                               directory.
  --default-artifact-root URI  Directory in which to store artifacts for any
                             

Command to configure tracking server is:

`mlflow server --backend-store-uri "sqlite:///mlflow.db" --default-artifact-root "./artifacts"`

Answer:
`--default-artifact-root`

## Q5. Tune the hyperparameters of the model
Now let's try to reduce the validation error by tuning the hyperparameters of the random forest regressor using hyperopt.

We have prepared the script `hpo.py` for this exercise.

Your task is to modify the script `hpo.py` and make sure that the validation RMSE is logged to MLflow for each run of the hyperparameter optimization (you will need to add a few lines of code to the objective function) and run the script without passing any parameters.

After that, open the MLflow UI and explore the runs from the experiment called random-forest-hyperopt to answer the question below.

__Important:__ don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

- the list of hyperparameters that are passed to the objective function during the optimization.
- the RMSE obtained on the validation set (February 2021 data).

What's the best validation RMSE that you got?

- 6.128
- 6.628
- 7.128
- 7.628

In [20]:
experiment = client.get_experiment_by_name("random-forest-hyperopt")

In [21]:
best_run = client.search_runs(
    experiment_ids=experiment.experiment_id, 
    run_view_type=mlflow.entities.ViewType.ACTIVE_ONLY,
    max_results=1,
    order_by=["metrics.rmse ASC"]
)[0]

In [22]:
best_run

<Run: data=<RunData: metrics={'rmse': 6.6284257482044735}, params={'max_depth': '19',
 'min_samples_leaf': '3',
 'min_samples_split': '5',
 'n_estimators': '28',
 'random_state': '42'}, tags={'mlflow.source.git.commit': '8cd4ade0854aeb907d3805ae97d2df7e5af60c00',
 'mlflow.source.name': 'hpo.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'nicolasroldan',
 'model': 'random-forest'}>, info=<RunInfo: artifact_uri='./mlruns/3/51d916c9f97f4fdfacd95e3267596a7f/artifacts', end_time=1654120973278, experiment_id='3', lifecycle_stage='active', run_id='51d916c9f97f4fdfacd95e3267596a7f', run_uuid='51d916c9f97f4fdfacd95e3267596a7f', start_time=1654120956785, status='FINISHED', user_id='nicolasroldan'>>

In [ ]:
dir(best_run)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_data',
 '_get_properties_helper',
 '_info',
 '_properties',
 'data',
 'from_dictionary',
 'from_proto',
 'info',
 'to_dictionary',
 'to_proto']

In [31]:
best_run.data.metrics

{'rmse': 6.6284257482044735}

In [36]:
print(f"Best validation RMSE is {best_run.data.metrics['rmse']}")

Best validation RMSE is 6.6284257482044735


## Q6. Promote the best model to the model registry

The results from the hyperparameter optimization are quite good so we can assume that we are ready to test some of these models in production. In this exercise, you'll promote the best model to the model registry. We have prepared a script called register_model.py, which will check the results from the previous step and select the top 5 runs. After that, it will calculate the RMSE of those models on the test set (March 2021 data) and save the results to a new experiment called random-forest-best-models.

Your task is to update the script register_model.py so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tip 1: you can use the method search_runs from the MlflowClient to get the model with the lowest RMSE. Tip 2: to register the model you can use the method mlflow.register_model and you will need to pass the right model_uri in the form of a string that looks like this: "runs:/<RUN_ID>/model", and the name of the model (make sure to choose a good one!).

What is the test RMSE of the best model?
- 6.1
- 6.55
- 7.93
- 15.1

After executing the following script:

In [ ]:
!python register_model.py --data_path ../processed-data

In [37]:
experiment = client.get_experiment_by_name("random-forest-best-models")

In [40]:
best_run_test_set = client.search_runs(
    experiment_ids=experiment.experiment_id, 
    max_results=1,
    run_view_type= mlflow.entities.ViewType.ACTIVE_ONLY,
    order_by=["metrics.test_rmse ASC"]
)[0]
print(f"Test RMSE for the best model is: {best_run_test_set.data.metrics['test_rmse']}")

Test RMSE for the best model is: 6.547886185595423
